<a href="https://colab.research.google.com/github/natalyasharygina/123/blob/master/ABTestSem8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**A/B-тестирование (семинары)**

***Домашняя работа к семинару 8***

Задание: На сайте запущен А/В тест с целью увеличить доход. В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [1]:
from tqdm import tqdm # Загрузка библиотек
import pandas as pd
import numpy as np
from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats.power import tt_ind_solve_power
import warnings
import plotly.express as px
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [4]:
from google.colab import files # Загрузка файла
uploaded = files.upload()

Saving AB_Test_Results.xlsx to AB_Test_Results.xlsx


In [8]:
df = pd.read_excel('AB_Test_Results.xlsx')
df.shape

(10000, 3)

In [9]:
df = df.groupby(['USER_ID', 'VARIANT_NAME']).agg({'REVENUE':'sum'}) # Чиска данных
df.reset_index(inplace=True)
df.drop_duplicates('USER_ID', keep=False, inplace=True)
df.shape

(4783, 3)

In [10]:
df.describe() # Первичный анализ

,USER_ID,REVENUE
count,4783.000000,4783.000000
mean,4994.395777,0.135873
std,2898.618472,3.011392
min,2.000000,0.000000
25%,2476.000000,0.000000
50%,4975.000000,0.000000
75%,7515.000000,0.000000
max,9998.000000,196.010000


In [11]:
control = df[df['VARIANT_NAME'] == 'control']
treatment = df[df['VARIANT_NAME'] == 'variant']
print(f"Контрольная группа: {control.count()[0]} Тестовая группа: {treatment.count()[0]}")

Контрольная группа: 2390 Тестовая группа: 2393


In [12]:
fig = px.histogram(df,
                   x='REVENUE',
                   color = 'VARIANT_NAME',
                   title='Revenue distribution',
                   marginal = 'box',
                   nbins = 100,
                   barmode='overlay')

fig.show()      # Визуализация

In [13]:
# Статические критерии
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам
    size = control.loc[:, column].shape[0]

    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()

    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)

    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = control.loc[:, column].sample(n=size, replace=True).values
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values
        booted_diff.append(np.mean(control_sample - treatment_sample))

    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))

    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci,
                           'beta': (1-power),
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,},
                          index=[column])
    return result

continious_result(control,
                  treatment,
                  column='REVENUE')

100%|██████████| 10000/10000 [00:05<00:00, 1972.70it/s]


,effect_size,alpha,beta,CI,difference
REVENUE,-0.040483,0.160821,0.712143,"[-0.004, 0.315]",0.120415


Рекомендации
Ноль лежит в пределах доверительного интервала, alpha намного больше 0.05, beta стремится к единице. На основе этого можно сделать выводы, что у нас нет статистически значимых данных. Следовательно, мы не можем выкатывать тестовое решение.

Необходимо искать другие решения для дальнейшего A/B-тестирования.